In [16]:
import pandas as pd
import jupyter as jp
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from pathlib import Path
from datetime import datetime
from datetime import timedelta

response = requests.post('https://oauth.battle.net/token', auth=('', ''), data={'grant_type': 'client_credentials'})
if response.status_code == 200:
    access_token = response.json()['access_token']
    print('Access token:', access_token)
else:
    print('Failed to retreive access token')
    print("status code:", response.status_code)
    print('Response:', response.text)

## Get list of Realms

In [11]:
access_token = "USpJZEkuqyNQmsXqh0mv0l1rQ7VvhINYiU"  # Replace with your actual access token

api_url = "https://us.api.blizzard.com/data/wow/realm/index"
namespace = "dynamic-us"  # Adjust as necessary
locale = "en_US"  # Locale can be adjusted as needed

headers = {
    'Authorization': f'Bearer {access_token}',
    'Battlenet-Namespace': namespace,
    'locale': locale
}

response = requests.get(api_url, headers=headers)

if response.status_code == 200:
    realms_data = response.json()
    list_of_realms=[]
    for realm in realms_data.get('realms', []):
        list_of_realms.append((realm['id'], realm['name']['en_US']))
    list_of_realms.sort()
else:
    print('Failed to retrieve realm data')
    print('Status Code:', response.status_code)
    print('Response:', response.text)

In [12]:
 list_of_realms

[(1, 'Lightbringer'),
 (2, 'Cenarius'),
 (3, 'Uther'),
 (4, 'Kilrogg'),
 (5, 'Proudmoore'),
 (6, 'Hyjal'),
 (7, 'Frostwolf'),
 (8, "Ner'zhul"),
 (9, "Kil'jaeden"),
 (10, 'Blackrock'),
 (11, 'Tichondrius'),
 (12, 'Silver Hand'),
 (13, 'Doomhammer'),
 (14, 'Icecrown'),
 (15, 'Deathwing'),
 (16, "Kel'Thuzad"),
 (47, 'Eitrigg'),
 (51, 'Garona'),
 (52, 'Alleria'),
 (53, 'Hellscream'),
 (54, 'Blackhand'),
 (55, 'Whisperwind'),
 (56, 'Archimonde'),
 (57, 'Illidan'),
 (58, 'Stormreaver'),
 (59, "Mal'Ganis"),
 (60, 'Stormrage'),
 (61, "Zul'jin"),
 (62, 'Medivh'),
 (63, 'Durotan'),
 (64, 'Bloodhoof'),
 (65, 'Khadgar'),
 (66, 'Dalaran'),
 (67, 'Elune'),
 (68, 'Lothar'),
 (69, 'Arthas'),
 (70, 'Mannoroth'),
 (71, 'Warsong'),
 (72, 'Shattered Hand'),
 (73, 'Bleeding Hollow'),
 (74, 'Skullcrusher'),
 (75, 'Argent Dawn'),
 (76, 'Sargeras'),
 (77, 'Azgalor'),
 (78, 'Magtheridon'),
 (79, 'Destromath'),
 (80, 'Gorgonnash'),
 (81, 'Dethecus'),
 (82, 'Spinebreaker'),
 (83, 'Bonechewer'),
 (84, 'Dragonmaw'

## Check if csv file exists

In [3]:
from pathlib import Path
csv_file = Path("auction_data.csv")
if csv_file.is_file(): csv_exists=True
else: csv_exists=False    

## Get Current Active Auctions

In [53]:
# import requests
# import pandas as pd
# from pandas import json_normalize
total_realms=len(list_of_realms)+1
# access_token = "USpJZEkuqyNQmsXqh0mv0l1rQ7VvhINYiU"  # Replace with your actual access token
start_list_processing_time=datetime.now()
for realm in list_of_realms:  # this will return a tuple, need to specify [0] when using the id part
    total_realms-=1
    # realm_id = 58  # Replace with the actual realm ID
    namespace = 'dynamic-us'  # Namespace for US region

    # Corrected API URL format
    api_url = f'https://us.api.blizzard.com/data/wow/connected-realm/{realm[0]}/auctions' # realm is a tuple, need to specify [0] for the id part of it
    
    # Parameters
    params = {
        'namespace': namespace,
        'locale': 'en_US'  # Or your preferred locale
    }
    
    # Headers
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    this_realm_start=datetime.now()
    try:
        # Making the GET request
        #https://us.api.blizzard.com/data/wow/connected-realm/1146/auctions?namespace=dynamic-us&locale=en_US&access_token=USpJZEkuqyNQmsXqh0mv0l1rQ7VvhINYiU
        response = requests.get(api_url, headers=headers, params=params)
    except Exception as err:
        print(err)
    # Checking the response
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data, record_path=['auctions'])
        df['realm_id']=realm[0]
        df['time_stamp']=pd.to_datetime(datetime.now())

        # Save DataFrame to CSV
        if csv_exists:
            # df.to_csv('C:/Users/Robert/Desktop/Project_1/auction_data.csv', index=False)
            df.to_csv('auction_data.csv', mode='a',index=False,header=False)
        else:
            df.to_csv('auction_data.csv', index=False)
            csv_exists=True
        secs=(datetime.now()-this_realm_start)
        secs=secs.seconds              
        print(f"Added {len(df)} for Realm {realm[1]} to auction_data.csv, {total_realms} Realms to go. (Realm Processing Time {timedelta(seconds=secs)})")
    else:
        print('-- O,O '*10)
        print(f'Failed to retrieve auction data for realm {realm}')        
        print("Status Code:", response.status_code, 'Response:', response.text)
        print('-- O,O '*10)

    secs=(datetime.now()-start_list_processing_time)
    secs=secs.seconds              
    print(f"Total processing time {timedelta(seconds=secs)}")
    

Failed to retrieve auction data for realm (1, 'Lightbringer')
Status Code: 404
Response: {"code":404,"type":"BLZWEBAPI00000404","detail":"Not Found"}
Total processing time 0:00:00
Failed to retrieve auction data for realm (2, 'Cenarius')
Status Code: 404
Response: {"code":404,"type":"BLZWEBAPI00000404","detail":"Not Found"}
Total processing time 0:00:01
Failed to retrieve auction data for realm (3, 'Uther')
Status Code: 404
Response: {"code":404,"type":"BLZWEBAPI00000404","detail":"Not Found"}
Total processing time 0:00:02
Added 14619 for Realm Kilrogg to auction_data.csv, 245 Realms to go. (Processing Time 0:00:02)
Total processing time 0:00:04
Added 78137 for Realm Proudmoore to auction_data.csv, 244 Realms to go. (Processing Time 0:00:09)
Total processing time 0:00:13
Failed to retrieve auction data for realm (6, 'Hyjal')
Status Code: 404
Response: {"code":404,"type":"BLZWEBAPI00000404","detail":"Not Found"}
Total processing time 0:00:14
Failed to retrieve auction data for realm (7,

In [54]:
auction_data=pd.read_csv('auction_data.csv')

C:\Users\Fidel\AppData\Local\Temp\ipykernel_16928\1943373240.py:1: DtypeWarning: Columns (4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  auction_data=pd.read_csv('auction_data.csv')


In [ ]:
display(api_url,headers,params)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.isna().sum()

In [ ]:
df['item.pet_breed_id'].value_counts().sum()/len(df)

In [ ]:
df.loc[df['item.pet_breed_id']>0]

In [ ]:
df['bid'].value_counts().sum()

In [ ]:
bonus_list=df['item.bonus_lists']
for item in bonus_list:
    if isinstance(item,list):
        if(len(item)>max):
            max=len(item)
            print(max, item)

In [ ]:
max=0

In [ ]:
exploded_bonus_list

In [ ]:
df['item.bonus_lists'].value_counts()

In [ ]:
df.loc[df['item.bonus_lists']==9527]

In [ ]:
modifiers=pd.json_normalize(df['item.modifiers'])
modifiers[0]

In [ ]:
type_28=pd.json_normalize(modifiers[0])
type_28

In [ ]:
type_28.value_counts()

In [ ]:
modifiers[1]

In [ ]:
type_29=pd.json_normalize(modifiers[1])
type_29

In [ ]:
bonus_list=pd.json_normalize(df['item.bonus_lists'])
bonus_list_list=pd.json_normalize(bonus_list[3])
bonus_list_list

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('C:/Users/Robert/Desktop/Project_1/auction_data.csv')
# remove data that is nan
df = df.dropna()
# Filter for the item 'Brain Hacker'
# Note: Adjust the column name 'item_name' to the actual column name in your CSV that holds the item names
brain_hacker_df = df[df['item.id'] == '1263']

# Extract the auction prices
# Note: Replace 'price' with the actual column name in your CSV that holds the prices
prices = brain_hacker_df['buyout']

# Plotting the prices
plt.figure(figsize=(10, 6))
plt.plot(prices, marker='o')
plt.title('Auction Prices for Brain Hacker')
plt.xlabel('Auction Entry')
plt.ylabel('Price')
plt.grid(True)
plt.show()